In [1]:
import os
import re
import json
import time
import glob
import shutil
import random
import requests
import aiofiles
import aiohttp
import asyncio
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime


df = pd.read_excel(r"dianping_sid.xlsx")
df_li = df.values.tolist()
sid_li = [i [1] for i in df_li]


async def getPost(sid):

    url_web = 'http://m.dianping.com/shop/' + sid
    url_1 = f'https://mapi.dianping.com/mapi/msource/shop.bin?device_system=MACINTOSH&lat=0&lng=0&mtsiReferrer=pages%2Fdetail%2Fdetail%3FshopUuid%3D{sid}%26online%3D1%26shopuuid%3D{sid}%26shopId%3D{sid}%26pageName%3Dshop&online=1&pageName=shop&shopUuid={sid}&shopuuid={sid}&'
    url_2 = f'https://m.dianping.com/wxmapi/shop/rankinfo?device_system=MACINTOSH&shopUuid={sid}&showPos=tag&'
    headers={'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}
    
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url=url_1, headers=headers) as resp1, session.get(url=url_2, headers=headers) as resp2:
                if (resp1.status == 200) and (resp2.status == 200):
                    data = await resp1.json()
                    rank = await resp2.json()
                else:
                    print('webpage status not 200 error')
                    data='NA'
                
                if data != '':
                    content = {'url': url_web, 'sid': sid, 'data': data, 'rank': rank}
                    async with aiofiles.open(f"dianping/{sid}.json", mode="w", encoding="utf-8") as f:
                        await f.write(json.dumps(content, ensure_ascii=False))
                    print({'shop':content['url'], 'rate':content['data']['shopPowerRate']})    
                else:
                    print(sid, "restarts1")
                    await getPost(sid) 
                
    except Exception:
        print(sid, "restarts2")
        await getPost(sid)


async def getPosts(sid_li, start_id, end_id):
    sids = os.listdir("dianping")
    finised_sids = [filename[:-5] for filename in sids]
    tasks = [asyncio.create_task(getPost(sid)) for sid in sid_li[start_id: end_id + 1] if sid not in finised_sids]
    try:
        await asyncio.wait(tasks)
    except Exception:
        print(f"{start_id} to {end_id} is full")
        
        
def makedir(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    if not os.path.exists(directory):
        os.mkdir(directory)

      
def get_csv():
    df = pd.concat([pd.read_json(i, orient='index').T for i in sorted(glob.glob('dianping/*.json'))], ignore_index=True)
    df['name'] = df.data.str.get('name')
    df['branchName'] = df.data.str.get('branchName')
    df['shopPowerRate'] = df.data.str.get('shopPowerRate')
    df['scoreText'] = df.data.str.get('scoreText')
    df['priceText'] = df.data.str.get('priceText')
    df['voteTotal'] = df.data.str.get('voteTotal')
    df['cityName'] = df.data.str.get('cityName')
    df['regionName'] = df.data.str.get('regionName')
    df['address'] = df.data.str.get('address')
    df['recentBizTime'] = df.data.str.get('recentBizTime').str.get('title')
    df['rankShortName'] = df['rank'].str.get('rankInfo').str.get('rankShortName')
    df['rankings'] = df['rank'].str.get('rankInfo').str.get('rankings').astype('Int64')
    df['rankUrl'] = df['rank'].str.get('rankInfo').str.get('rankUrl')
    df['date'] = datetime.now().strftime('%Y-%m-%d')
    df = df.drop(columns=['data', 'rank']).replace(np.nan, np.nan).replace('', np.nan)
    df.to_csv(f"result_{datetime.now().strftime('%Y-%m-%d')}.csv", index=False)


if __name__ == '__main__':
    makedir("dianping")
    for i in tqdm(range(0, len(df_li), 100)):
        t2 = datetime.now()
        time.sleep(5)
        await getPosts(sid_li, i, i + 99)
        print("Num:", min(len(df_li), i + 100), "/", len(df_li), "ETA:", (datetime.now() - t2) / 100 * max(0, (len(df_li) - i - 100)))
    get_csv()
    print("Done!")

  0%|          | 0/41 [00:00<?, ?it/s]

{'shop': 'http://m.dianping.com/shop/H3xlVRxEi3Hh8Y4f', 'rate': '4.1'}
{'shop': 'http://m.dianping.com/shop/l1L6u7yTA9HVUzVS', 'rate': '4.2'}
{'shop': 'http://m.dianping.com/shop/la3dROlfjPH9OdBn', 'rate': '4.5'}
{'shop': 'http://m.dianping.com/shop/k9CztudsgD57MPOZ', 'rate': '3.8'}
{'shop': 'http://m.dianping.com/shop/k381s1PupZ92fSE6', 'rate': '4.3'}
{'shop': 'http://m.dianping.com/shop/G2Kj7q7Ed7X4fdr1', 'rate': '4.6'}
{'shop': 'http://m.dianping.com/shop/k9DSdrZjJuIXqIId', 'rate': '4.4'}
{'shop': 'http://m.dianping.com/shop/iMeOqCud3RjUlrV3', 'rate': '3.9'}
{'shop': 'http://m.dianping.com/shop/ka4i4xLvIFNWyFgX', 'rate': '4.9'}
{'shop': 'http://m.dianping.com/shop/l8024JHAnyi6Lafu', 'rate': '3.7'}
{'shop': 'http://m.dianping.com/shop/H9yc8oU9O8bCwUMN', 'rate': '4.7'}
{'shop': 'http://m.dianping.com/shop/jRE9jD8IxuVhB62o', 'rate': '4.9'}
{'shop': 'http://m.dianping.com/shop/Ha26sBWw7pxGZLfR', 'rate': '3.8'}
{'shop': 'http://m.dianping.com/shop/H6FGdoA0XFTq1OBH', 'rate': '3.9'}
{'shop

  2%|▏         | 1/41 [00:13<09:13, 13.84s/it]

{'shop': 'http://m.dianping.com/shop/k32y9yG5u57v0vyC', 'rate': '4.0'}
Num: 100 / 4053 ETA: 0:09:07.150542
{'shop': 'http://m.dianping.com/shop/l4AXmohZbe0rde9J', 'rate': '3.9'}
{'shop': 'http://m.dianping.com/shop/k9bM2Rd2by1Chucj', 'rate': '4.0'}
{'shop': 'http://m.dianping.com/shop/G1TJa5IS81CZUG7U', 'rate': '4.0'}
{'shop': 'http://m.dianping.com/shop/k6626AzLsci3b2xd', 'rate': '3.4'}
{'shop': 'http://m.dianping.com/shop/k2Fm2rA8LlvfZQui', 'rate': '3.9'}
{'shop': 'http://m.dianping.com/shop/H9gXYejJnYWTWh9v', 'rate': '4.0'}
{'shop': 'http://m.dianping.com/shop/j16smalZzJLu1xhP', 'rate': '3.9'}
{'shop': 'http://m.dianping.com/shop/G2ofNc3wGuAmbS68', 'rate': '3.8'}
{'shop': 'http://m.dianping.com/shop/laVMsS7OWNYG8R5Y', 'rate': '4.2'}
{'shop': 'http://m.dianping.com/shop/l6lrFq0pQrTP7lWW', 'rate': '3.9'}
{'shop': 'http://m.dianping.com/shop/H6w2kxwwuiO68bcW', 'rate': '3.4'}
{'shop': 'http://m.dianping.com/shop/H5sPhpX8tt6vLbcg', 'rate': '4.0'}
{'shop': 'http://m.dianping.com/shop/H2yq

  5%|▍         | 2/41 [00:27<08:49, 13.58s/it]

...

 27%|██▋       | 11/41 [05:06<16:30, 33.00s/it]

{'shop': 'http://m.dianping.com/shop/G3nsC7d1fAUYjyLh', 'rate': '4.8'}
Num: 1100 / 4053 ETA: 0:21:30.396034
{'shop': 'http://m.dianping.com/shop/l1DSr75e1nGt3Rro', 'rate': '3.9'}
{'shop': 'http://m.dianping.com/shop/l41nIB1Ru54HqUql', 'rate': '4.5'}
{'shop': 'http://m.dianping.com/shop/k3cjKy1clcgVQC4K', 'rate': '4.9'}
{'shop': 'http://m.dianping.com/shop/H63IhwtEHGRterkb', 'rate': '4.1'}
{'shop': 'http://m.dianping.com/shop/k7iSPIXCWcRgYHTx', 'rate': '4.5'}
{'shop': 'http://m.dianping.com/shop/k44QOn2u03qvfQQH', 'rate': '4.8'}
{'shop': 'http://m.dianping.com/shop/l9DbFndXZpxNyHWz', 'rate': '3.9'}
{'shop': 'http://m.dianping.com/shop/EvoISeIC9NV4Nc3E', 'rate': '4.8'}
{'shop': 'http://m.dianping.com/shop/ZiFtkoaKuV7cXJHS', 'rate': '4.3'}
{'shop': 'http://m.dianping.com/shop/l7HGjSEknqSkPJb8', 'rate': '4.7'}
{'shop': 'http://m.dianping.com/shop/H9OVBO3yB3OBk89J', 'rate': '4.2'}
{'shop': 'http://m.dianping.com/shop/k4NK2SEPDgQSAhGA', 'rate': '4.0'}
{'shop': 'http://m.dianping.com/shop/k3T

 27%|██▋       | 11/41 [05:39<15:26, 30.89s/it]


CancelledError: 

k39nopYvItPaeZTp restarts2
